In [1]:
import pandas as pd

In [2]:
df_sent = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\sentiment_analysis_2.parquet', columns= ['item_id', 'sentiment_analysis_2', 'recommend', 'Año'])
df_steam = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\steam_games.parquet', columns= ['id', 'developer'])
df = df_sent.merge(df_steam, how='left', left_on='item_id', right_on='id')
df.drop(columns=['item_id', 'id'], inplace=True)
df.dropna(inplace=True)
df = df.groupby(['Año', 'developer']).agg({'sentiment_analysis_2': (lambda x: (x == 2).sum()), 'recommend': 'sum'})
df['rating'] = df['sentiment_analysis_2'] + df['recommend']
df.drop(columns=['sentiment_analysis_2', 'recommend'], inplace=True)
df.reset_index(inplace=True)
df.head()

,Año,developer,rating
0,0,07th Expansion,1
1,0,10tons Ltd,2
2,0,16bit Nights,1
3,0,22cans,4
4,0,"2K Boston,2K Australia,Blind Squirrel,Feral In...",10


In [3]:
df.to_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\best_developer_year.parquet')

In [6]:
def best_developer_year(anio: int):

    try:
        anio = int(anio)
    except Exception as e:
        return {f'Error {e}': 'Debe insertar un número entero.'}
    
    df = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\best_developer_year.parquet')
    anios = list(df['Año'].unique())
    anios = [int(x) for x in anios]
    
    df = df[df['Año'].isin([anio])]
    if len(df) == 0:
        del df
        return {'Mensaje': f'No hay registros del año {anio}',
                'Los años disponibles son:': anios}

    df = df.sort_values('rating', ascending=False).reset_index()
    
    result = {
                'Puesto 1': df.loc[0, 'developer'],
                'Puesto 2': df.loc[1, 'developer'],
                'Puesto 3': df.loc[2, 'developer']
            }
    del df
    return result

In [8]:
best_developer_year(2010)

{'Puesto 1': 'Valve',
 'Puesto 2': 'Tripwire Interactive',
 'Puesto 3': 'Facepunch Studios'}